In [473]:
import json
import os
from typing import Any, Dict, NamedTuple
import urllib

import requests

In [474]:
username = 'alysivji'

In [475]:
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')

In [476]:
_HEADERS = {
    'User-Agent': 'Aggregit',
    'Authorization': f'token {GITHUB_TOKEN}',
}
_GITHUB_API_URL = 'https://api.github.com'

In [477]:
class Response(NamedTuple):
    json: Dict[str, Any]
    headers: Dict[str, str]


## Helper Function

In [478]:
def _extract_page_from_link_header(link):
    """
    Helper function to extract page 
    """
    extracted_url = link.split('<')[1].split('>')[0]
    url_details = urllib.parse.urlparse(extracted_url)
    query_string = url_details.query
    params = urllib.parse.parse_qs(query_string)
    return int(params['page'][0])

assert _extract_page_from_link_header('<https://api.github.com/user/4369343/repos?type=all&page=2>; rel="last"') == 2

In [479]:
def github_api_get(endpoint, *, params={}):
    """
    Given GitHub endpoint, returns JSON response and headers
    """
    # remove leading slash from endpoint if it is included
    if endpoint[0] == '/':
        endpoint = endpoint[1:]
        
    r = requests.get(f'{_GITHUB_API_URL}/{endpoint}', headers=_HEADERS, params=params)
    
    return Response(r.json(), r.headers)

In [480]:
def get_all_items(endpoint):
    all_items = []

    # get items from first page
    resp = github_api_get(endpoint, params={'type': 'all', 'page': 1})
    r_json = resp.json
    r_headers = resp.headers
    all_items.extend(r_json)

    # if multiple pages, pull all items
    if r_headers.get('Link', None):
        for link in r_headers['Link'].split(', '):
            if 'rel="last"' in link:
                last_page = _extract_page(link)

        for page in range(2, last_page + 1):
            r_json, _ = github_api_get(endpoint, params={'type': 'all', 'page': page})

            all_items.extend(r_json)
            
    return all_items

## Get All Repos

In [481]:
all_repos_endpoint = f'users/{username}/repos'
all_repos = get_all_items(all_repos_endpoint)

### Sample Point

In [482]:
all_repos[0]

{'archive_url': 'https://api.github.com/repos/alysivji/20180215-chipy-project-night/{archive_format}{/ref}',
 'archived': False,
 'assignees_url': 'https://api.github.com/repos/alysivji/20180215-chipy-project-night/assignees{/user}',
 'blobs_url': 'https://api.github.com/repos/alysivji/20180215-chipy-project-night/git/blobs{/sha}',
 'branches_url': 'https://api.github.com/repos/alysivji/20180215-chipy-project-night/branches{/branch}',
 'clone_url': 'https://github.com/alysivji/20180215-chipy-project-night.git',
 'collaborators_url': 'https://api.github.com/repos/alysivji/20180215-chipy-project-night/collaborators{/collaborator}',
 'comments_url': 'https://api.github.com/repos/alysivji/20180215-chipy-project-night/comments{/number}',
 'commits_url': 'https://api.github.com/repos/alysivji/20180215-chipy-project-night/commits{/sha}',
 'compare_url': 'https://api.github.com/repos/alysivji/20180215-chipy-project-night/compare/{base}...{head}',
 'contents_url': 'https://api.github.com/repos/

---

In [318]:
repo_names = [item['name'] for item in all_repos]
repo_urls = [item['url'] for item in all_repos]

## Calculate Stats

In [242]:
originals = [item['name'] for item in all_repos if item['fork'] is False]
forks = [item['name'] for item in all_repos if item['fork'] is True]

assert len(originals) + len(forks) == len(all_repos)

In [322]:
print(f'Total Num of Original Repos: {len(originals)}')
print(f'Total Num of Forked Repos: {len(forks)}')

Total Num of Original Repos: 38
Total Num of Forked Repos: 13


In [319]:
all_followers_endpoint = f'users/{username}/followers'
followers = get_all_items(all_followers_endpoint)

In [323]:
print(f'Total Num of Followers: {len(followers)}')

Total Num of Followers: 55


In [324]:
all_watchers_endpoint = f'users/{username}/subscriptions'
watchers = get_all_items(all_watchers_endpoint)

In [333]:
print(f'Total Num of Repos Watched: {len(watchers)}')

Total Num of Repos Watched: 8


In [347]:
stars_recieved = sum([item['stargazers_count'] for item in all_repos])

In [349]:
print(f'Total Num of Stars Recieved: {stars_recieved}')

Total Num of Stars Recieved: 32


In [350]:
all_stars_given = f'users/{username}/starred'
stars_given = get_all_items(all_stars_given)

In [354]:
print(f'Total Num of Stars Recieved: {len(stars_given)}')

Total Num of Stars Recieved: 310


In [355]:
repo_sizes = [item['size'] for item in all_repos]
total_size = sum(repo_sizes)

In [357]:
print(f'Total Size of Account: {total_size}')

Total Size of Account: 293297


In [359]:
open_issues_list = [item['open_issues_count'] for item in all_repos if item['fork'] is False]
open_issues = sum(open_issues_list)

In [360]:
print(f'Total Number of Open Issues in Repos: {open_issues}')

Total Number of Open Issues in Repos: 2


### Number of Commits

In [395]:
original_repos = [item['commits_url'] for item in all_repos if item['fork'] is False]

In [423]:
repo_commit_endpoints = [repo.split('{/sha}')[0].split('https://api.github.com/')[1]
                         for repo in original_repos]

In [406]:
commits = []

for repo in repo_commit_endpoints:
    repo_commits = get_all_items(repo)
    commits.extend(repo_commits)

https://api.github.com/repos/alysivji/20180215-chipy-project-night/commits
https://api.github.com/repos/alysivji/20180315-chipy-project-night/commits
https://api.github.com/repos/alysivji/advent-of-code-2017/commits
https://api.github.com/repos/alysivji/alysivji.github.io/commits
https://api.github.com/repos/alysivji/alysivji.github.io/commits
https://api.github.com/repos/alysivji/alysivji.github.io/commits
https://api.github.com/repos/alysivji/area-code-alexa-skill/commits
https://api.github.com/repos/alysivji/blog/commits
https://api.github.com/repos/alysivji/blog/commits
https://api.github.com/repos/alysivji/blog-notebooks/commits
https://api.github.com/repos/alysivji/chipy-python-project-night-prompt/commits
https://api.github.com/repos/alysivji/chipy-python-project-night-prompt/commits
https://api.github.com/repos/alysivji/chipy-python-project-night-prompt/commits
https://api.github.com/repos/alysivji/codeeveryday-hashtag-dashboard/commits
https://api.github.com/repos/alysivji/con

In [410]:
print(f'Total Number of Commits in public (non-forked) repos: {len(commits)}')

Total Number of Commits in public (non-forked) repos: 559


### Languages

In [446]:
repo_language_endpoints = [repo['languages_url'] for repo in all_repos]

In [449]:
repo_languages = []
for repo_endpoint in repo_language_endpoints:
    r = requests.get(repo_endpoint, headers=_HEADERS)
    repo_languages.extend(list(r.json().keys()))

In [452]:
print(f'Repo languages: {list(set(repo_languages))}')

Repo languages: ['C++', 'Shell', 'Smarty', 'Mako', 'Jupyter Notebook', 'Ruby', 'HTML', 'CSS', 'Vim script', 'Python', 'C', 'JavaScript', 'Makefile', 'Batchfile', 'PowerShell']


### Topics

In [483]:
topic_headers = dict(_HEADERS)
topic_headers['Accept'] = 'application/vnd.github.mercy-preview+json'

In [484]:
repo_topics = []

for repo in all_repos:
    repo_topic_url = f"{repo['url']}/topics"
    r = requests.get(repo_topic_url, headers=topic_headers)
    
    repo_topics.extend(r.json()['names'])

In [485]:
print(f'Repo topics: {list(set(repo_topics))}')

Repo topics: ['reddit', 'scrapy', 'mongodb', 'scraping']
